In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [2]:
from glob import glob

In [4]:
files = glob('url/*.json')
len(files)

2610

In [23]:
urls = []
for f in files:
    with open(f) as fopen:
        urls.extend(json.load(fopen))
        
urls = list(set(urls))
urls = sorted(['https://technave.com/' + u for u in urls])
len(urls)

25084

In [24]:
def crawl(url, filename):
    if os.path.exists(filename):
        return True
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    try:
        title = soup.find('h1').text
    except:
        title = None
    try:
        p = soup.find('div', {'class': 'default'}).find_all('p')
        p = [p_.text for p_ in p]
    except:
        p = None
    
    data = {
        'url': url,
        'title': title,
        'p': p,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

In [25]:
# !rm -rf page
!mkdir page

mkdir: cannot create directory ‘page’: File exists


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for i in tqdm(range(0, len(urls), max_worker)):
    aranged = np.arange(i, i + max_worker)
    urls_ = [(urls[a], os.path.join('page', f'{a}.json')) for a in aranged]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url[0], url[1]): url for url in urls_}

    for future in as_completed(futures):
        future.result()

 44%|███████████████▏                   | 1093/2509 [1:44:29<2:33:43,  6.51s/it]